# **LLM Zoomcamp 2024 Competition Notebook**



In [1]:
# Install necessary packages
! pip install -qU langchain groq langchain-groq

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-cv 0.9.0 requires keras-core, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 16.1.0 which is incompatible.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.3, but you have packaging 24.1 which is incompatible.
jupyterlab 4.2.3 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.p

In [2]:
import pandas as pd

In [3]:
import os
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
groq_key = user_secrets.get_secret("GROQ_API_KEY")
os.environ["GROQ_API_KEY"] = groq_key


In [4]:
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate

from langchain_core.rate_limiters import InMemoryRateLimiter

rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.5, 
    check_every_n_seconds=0.25
)

llm = ChatGroq(
    model="llama3-groq-70b-8192-tool-use-preview",
    temperature=0,
    max_tokens=None,
    timeout=15,
    max_retries=5,
    rate_limiter=rate_limiter
    # other params...
)

/tmp/ipykernel_18/2539070966.py:6: LangChainBetaWarning: Introduced in 0.2.24. API subject to change.
  rate_limiter = InMemoryRateLimiter(


#### Function to query the LLM

In [5]:
def get_answer(question):
    prompt = f"""Role:
You are an advanced AI system with exceptional mathematical reasoning and problem-solving capabilities, specifically designed to solve tricky math problems (whose answer is a non-negative integer) written in LaTeX format from the AI Mathematical Olympiad (AIMO) competition. Your task is to accurately analyze and solve intricate mathematical problems, demonstrating a deep understanding of mathematical concepts and a strong ability to apply logical reasoning strategies.

Instruction:
1. Carefully read and comprehend the problem statement provided in the "Problem" section.
2. In the "Solution" section, provide a solution of the problem with detailed explanation of your logical reasoning process. Keep in mind that answer must be a non-negative integer number.
3. At the end, create a "Answer" section where you will state only the final numerical or algebraic answer, without any additional text or narrative.

Problem:
...

Solution:
...

Answer:
...

{question}

Step-by-step solution and final answer:"""

    response = llm.invoke(prompt)
    return response.content.strip()

#### Function to Extract Answer with

In [6]:
import re

def extract_numerical_answer(text):
    # Look for patterns like "Final answer: X" or "The answer is X" at the end of the text
    match = re.search(r'(?:final answer|the answer is)[:\s]*([+-]?\d*\.?\d+)', text, re.IGNORECASE)
    if match:
        return float(match.group(1))
    else:
        # If no clear final answer, look for the last number in the text
        numbers = re.findall(r'[+-]?\d*\.?\d+', text)
        return float(numbers[-1]) if numbers else None

#### Function to Prepare Dataset and Get Answers with

In [7]:
from tqdm.auto import tqdm

from concurrent.futures import ThreadPoolExecutor

pool = ThreadPoolExecutor(max_workers=4)

def map_progress(pool, seq, f):
    results = []

    with tqdm(total=len(seq)) as progress:
        futures = []

        for el in seq:
            future = pool.submit(f, el)
            future.add_done_callback(lambda p: progress.update())
            futures.append(future)

        for future in futures:
            result = future.result()
            results.append(result)

    return results

In [8]:
def process_row(row):
    problem_id = row['problem_id']
    problem_text = row['problem_text']

    llm_reasoning = get_answer(problem_text)

    numerical_answer = extract_numerical_answer(llm_reasoning)

    return {
        'problem_id': problem_id,
        'problem_text': problem_text,
        'llm_reasoning': llm_reasoning,
        'answer': numerical_answer
    }

In [9]:
df_train = pd.read_csv('/kaggle/input/llm-zoomcamp-2024-competition/train.csv')
df_train.head()

,problem_id,problem_text,answer
0,2374,Find the value of the expression $\dfrac{17}{5...,1.6
1,4723,"In a company of 30 people, 25 use the social n...",24
2,7135,The number of road traffic accidents (RTAs) in...,32
3,5814,Find the value of the expression $\dfrac{2\str...,256
4,9237,A traveler from Moscow wants to visit four cit...,53


In [10]:
rows = df_train.head().to_dict(orient='records')
process_row(rows[0])

{'problem_id': 2374,
 'problem_text': 'Find the value of the expression $\\dfrac{17}{5} :\\dfrac{34}{3} +1.3$.',
 'llm_reasoning': "Solution:\nTo solve this problem, we need to follow the order of operations (PEMDAS/BODMAS), which means we should perform calculations in the following order: Parentheses, Exponents, Multiplication and Division (from left to right), and Addition and Subtraction (from left to right).\n\nThe given expression is $\\dfrac{17}{5} :\\dfrac{34}{3} +1.3$. First, let's simplify the fractions:\n\n$$\\dfrac{17}{5} = 3.4$$\n\n$$\\dfrac{34}{3} = 11.33\\ldots$$\n\nNow, we have the expression:\n\n$$3.4 : 11.33\\ldots + 1.3$$\n\nNext, we need to perform division:\n\n$$3.4 : 11.33\\ldots = 0.3$$\n\nFinally, we add the remaining numbers:\n\n$$0.3 + 1.3 = 1.6$$\n\nAnswer:\n1.6",
 'answer': 1.6}

In [11]:
results = map_progress(pool, rows, process_row)
df_results = pd.DataFrame(results)
df_results

  0%|          | 0/5 [00:00<?, ?it/s]

,problem_id,problem_text,llm_reasoning,answer
0,2374,Find the value of the expression $\dfrac{17}{5...,"Solution:\nTo solve this problem, we need to f...",1.6
1,4723,"In a company of 30 people, 25 use the social n...","Solution:\n\n1. In this company, there will be...",24.0
2,7135,The number of road traffic accidents (RTAs) in...,Solution:\nLet's denote the number of road tra...,32.0
3,5814,Find the value of the expression $\dfrac{2\str...,"Solution:\nTo solve this problem, we will use ...",256.0
4,9237,A traveler from Moscow wants to visit four cit...,Solution:\nTo find the optimal combination of ...,46.0


In [12]:
df_train.head()

,problem_id,problem_text,answer
0,2374,Find the value of the expression $\dfrac{17}{5...,1.6
1,4723,"In a company of 30 people, 25 use the social n...",24
2,7135,The number of road traffic accidents (RTAs) in...,32
3,5814,Find the value of the expression $\dfrac{2\str...,256
4,9237,A traveler from Moscow wants to visit four cit...,53


In [13]:
import pandas as pd
import numpy as np


class ParticipantVisibleError(Exception):
    # If you want an error message to be shown to participants, you must raise the error as a ParticipantVisibleError
    # All other errors will only be shown to the competition host. This helps prevent unintentional leakage of solution data.
    pass


def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str) -> float:
    '''
    Accuracy that works with multiple correct answers.
    '''
    solution = solution.set_index(row_id_column_name, drop=True)
    submission = submission.set_index(row_id_column_name, drop=True)
    submission = submission.loc[solution.index]
        
    target_column = 'answer'
    assert target_column in solution.columns
    assert target_column in submission.columns

    # This fix is needed because submission is loaded with default parameters
    # Pandas magically converts string column into float
    def fix_suffix(value):
        if value.endswith('.0'):
            return value[:-2]
        else:
            return value
        
    submission[target_column] = submission[target_column].astype(str)
    submission[target_column] = submission[target_column].apply(fix_suffix)
    
    
    def convert_to_list(value):
        values = [v.strip() for v in value.strip().lstrip('[').rstrip(']').split(',')]
        return values

    solution[target_column] = solution[target_column].astype(str).apply(convert_to_list)

    correct = [
        submit_answer in correct_answer
        for correct_answer, submit_answer in zip(
            solution[target_column].values, 
            submission[target_column].values
        )
    ]
            
    return np.mean(correct)

In [14]:
score(df_train.head(), df_results, 'problem_id')

0.8

In [15]:
def prepare_prompts_and_get_answers(df):
    rows = df.to_dict(orient='records')
    results = map_progress(pool, rows, process_row)
    return pd.DataFrame(results)

In [16]:
df_train_results = prepare_prompts_and_get_answers(df_train)
df_train_results.head()

  0%|          | 0/100 [00:00<?, ?it/s]

,problem_id,problem_text,llm_reasoning,answer
0,2374,Find the value of the expression $\dfrac{17}{5...,"Solution:\nTo solve this problem, we need to f...",1.6
1,4723,"In a company of 30 people, 25 use the social n...","Solution:\n\n1. In this company, there will be...",24.0
2,7135,The number of road traffic accidents (RTAs) in...,Solution:\nLet's denote the number of road tra...,32.0
3,5814,Find the value of the expression $\dfrac{2\str...,"Solution:\nTo solve this problem, we will use ...",256.0
4,9237,A traveler from Moscow wants to visit four cit...,Solution:\nTo find the optimal combination of ...,46.0


In [17]:
score(df_train, df_train_results, 'problem_id')

0.52

## Preparing the test submission

In [18]:
df_test = pd.read_csv('/kaggle/input/llm-zoomcamp-2024-competition/test.csv')

df_test_results = prepare_prompts_and_get_answers(df_test)

submission = df_test_results[['problem_id', 'answer']]
submission.to_csv('submission.csv', index=False)

  0%|          | 0/100 [00:00<?, ?it/s]